# MSc Thesis 
# Jacotte Monroe

Code that retrieves automatically the MODIS and Landsat scenes of interest and loads them. 

## Importing and initializing 

In [1]:
import ee
ee.Authenticate()

True

In [2]:
import ee
import geemap
import os
import math

ee.Initialize()

## Set-up

In [3]:
# define dates 
start_date = ee.Date('2014-01-14') # '2014-02-15'
end_date = ee.Date('2014-02-18') # '2012-02-17'

In [4]:
# set time-window for interpolation (how far will interpolate) --> NO INTERPOLATION DONE IN THIS CODE BUT JUST IN CASE IN FUTURE
# source: https://spatialthoughts.com/2021/11/08/temporal-interpolation-gee/
days = ee.Number(10) 

# convert to milliseconds (for gap filling step)
millis = days.multiply(1000*60*60*24)

In [5]:
# load Etosha National Park study area
enp = ee.FeatureCollection('WCMC/WDPA/current/polygons') \
        .filter(ee.Filter.eq('ORIG_NAME', 'Etosha'))

# turn ENP study area into a geometry
enp_geom = enp.geometry()

In [6]:
# define sample study area 
# note: sample area is smaller than original sample area because of Landsat data size limit when exporting 
sample_area = ee.Geometry.Polygon(
        [[[15.457215604955591, -18.68807168881119],
          [15.457215604955591, -19.120709491695354],
          [15.939240751439966, -19.120709491695354],
          [15.939240751439966, -18.68807168881119]]], None)

## Dataset retrieval

In [7]:
## Landsat 8 
# note: ee.List should also normally include 180 (removed it for this sampled study area)
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2008-10-01', '2014-10-01') \
    .filter(ee.Filter.inList('WRS_PATH', ee.List([179]))) \
    .filter(ee.Filter.eq('WRS_ROW', 73)) \
    .select(['SR_B.'])

# remove B1 from Landsat 8 image collection 
# source: https://developers.google.com/earth-engine/apidocs/ee-list-remove
l8 = l8.select(ee.List(l8.first().bandNames().getInfo()).remove('SR_B1'))

# retrieve L8 dates 
# source: https://developers.google.com/earth-engine/apidocs/ee-imagecollection-aggregate_array
#l8_dates = l8.aggregate_array('DATE_ACQUIRED')
# source: https://gis.stackexchange.com/questions/307115/earth-engine-get-dates-from-imagecollection
# source (python API): https://developers.google.com/earth-engine/guides/python_install#syntax
#l8_dates = l8_dates.map(lambda x: ee.Date(x).advance(1, 'day').format('YYYY_MM_dd'))

#print('L8 dates', l8_dates.getInfo())

In [8]:
## MODIS daily 250m
# take larger time range to include images if I want gap filling of cloudmasked pixels 
# source: https://developers.google.com/earth-engine/apidocs/ee-date-advance#colab-python
#alternative to filterDate: filter(ee.Filter.inList('system:index', l8_dates)) \
modis_250 = ee.ImageCollection('MODIS/061/MOD09GQ') \
    .filterDate(start_date.advance(days.multiply(-1), 'day'), end_date.advance(days, 'day')) \
    .filterBounds(sample_area) \

# import MODIS 500m dataset (contains information for cloudmasking)
modis_500 = ee.ImageCollection('MODIS/006/MOD09GA') \
        .filterDate(start_date.advance(days.multiply(-1), 'day'), end_date.advance(days, 'day'))  \
        .filterBounds(sample_area)

In [9]:
# combine datasets
# pixel with certain MODIS 500m value can be selected for MODIS 250m 
# source: https://gis.stackexchange.com/questions/308456/get-a-mask-for-modis-250m-mod09gq-using-modis-500m-mod09ga-in-google-earth
modis_combined = modis_500.combine(modis_250)

# new name of MODIS 250m bands (to avoid having two of similar name)
modisBands = ['sur_refl_b01_1', 'sur_refl_b02_1']
renamedBands = ['red', 'nir']

## Functions

In [10]:
# function to extract and combined relevant QA band bits/flags 
# source: https://mygeoblog.com/2017/09/08/modis-cloud-masking/
def getQABits(image, start_bit, end_bit, band_name):
    pattern = 0
    
    # for each bit/flag of the QA band --> assign new value to its bits 
    # so each flag will have a different value and the pixel will be the sum of all flags 
    for i in range(start_bit, end_bit):
        pattern += math.pow(2,i)
    # return single band image of extracted QA bits
    # source: https://developers.google.com/earth-engine/apidocs/ee-image-bitwiseand
    return image.select([0], [band_name]) \
                .bitwiseAnd(pattern) \
                .rightShift(start_bit)

In [11]:
# function to mask out cloudy pixels 
# source: https://gis.stackexchange.com/questions/308456/get-a-mask-for-modis-250m-mod09gq-using-modis-500m-mod09ga-in-google-earth
# source: https://mygeoblog.com/2017/09/08/modis-cloud-masking/
def maskClouds(image):
    # selects the MODIS 500m QA band 
    QA = image.select('state_1km')

    # creates a cloud&shadow flag from specified bits --> in this case: 'Cloud state' and 'Cloud shadow'
    pixelQuality = getQABits(QA, 0, 2, 'cloud_and_shadow_quality_flag')

    # returns image masking out cloudy pixels 
    return image.updateMask(pixelQuality.eq(0))

In [12]:
# function to reproject (32733 if want Namibia coordinates, but elephant fixes are in 4326)
def reprojectLandsat(image): 
    return image.reproject('EPSG:4326', None, 30)

def reprojectModis(image):
    return image.reproject('EPSG:4326', None, 250)


In [13]:
# function to clip image to study area
def clipToAOI(image): 
    result = image.clip(sample_area)
    return result.copyProperties(image, ['system:id'])

In [14]:
# define landsat scaling factor
def applyScaleFactors(image): 
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    return image.addBands(opticalBands, None, True)

In [15]:
# function to calculate NDVI 
def addNDVI(image): 
    ndvi = image.normalizedDifference(['nir', 'red']).rename('NDVI')
    return image.addBands(ndvi)

## Cloudmasking MODIS images

In [16]:
# create cloud free MODIS 250m dataset 
# source: https://mygeoblog.com/2017/09/08/modis-cloud-masking/ 
# source: https://gis.stackexchange.com/questions/308456/get-a-mask-for-modis-250m-mod09gq-using-modis-500m-mod09ga-in-google-earth

modis_cloudFree = modis_combined.map(maskClouds).select(modisBands, renamedBands)

## Apply functions

In [17]:
# apply reprojection 
modis_reproj = modis_cloudFree.map(reprojectModis)
l8_reproj = l8.map(reprojectLandsat)

In [18]:
# apply clipping function 
modis_clipped = modis_reproj.map(clipToAOI)
l8_clipped = l8_reproj.map(clipToAOI)

In [19]:
# apply scaling factor 
l8_scaled = l8_clipped.map(applyScaleFactors)

# is this function actually working? because when i map this it doesn't match visualization parameters 
    # for scaled image instead it matches for non scaled image

In [20]:
# calculate NDVI for MODIS
modis_ndvi = modis_clipped.map(addNDVI)

## Visualize

In [18]:
# define visualization parameters
visualization_modis = {
  'min': -100.0,
  'max': 8000.0,
  'bands': ['nir', 'nir', 'red'],
}

visualization_l8 = {
  'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
  'min': 8000,
  'max': 20000,
}

visualization_l8_scaled = {
  'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
  'min': 0,
  'max': 0.5,
}

In [22]:
# initialization and set up of the map
Map = geemap.Map()
#Map.add_ee_layer(enp, None, 'Etosha National Park')
#Map.add_layer(sample_area, None, 'Sample AOI')
#Map.add_layer(l8_scaled.first(), visualization_l8_scaled, 'L8 Scaled')
modis = modis_clipped.filterDate('2014-01-19', '2014-01-20')
Map.add_layer(modis_cloudFree.filterDate('2014-01-19', '2014-01-20'), visualization_modis, 'MODIS Cloud Free Jan 19')
Map.add_layer(modis, visualization_modis, 'MODIS final Jan 19')
Map.centerObject(enp_geom)

# Show the map
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Export data

In [21]:
# get output folder path 
# source: https://github.com/gee-community/geemap/blob/master/examples/notebooks/11_export_image.ipynb
out_dir_l8 = os.path.join(os.path.expanduser('~'), 'Documents/MSc_Thesis/data/l8')
out_dir_m = os.path.join(os.path.expanduser('~'), 'Documents/MSc_Thesis/data/modis/cloudmasked')

In [22]:
# export Landsat image collection to local repository 
# source: https://github.com/gee-community/geemap/blob/master/examples/notebooks/11_export_image.ipynb
#geemap.ee_export_image_collection(l8_clipped, out_dir = out_dir_l8)

In [23]:
# filter MODIS dates to only take dates of interest, exclude images that were used for gap filling (if gap filled)
modis = ee.ImageCollection(modis_ndvi).filterDate(start_date, end_date)

# export MODIS image collection to local repository 
# source: https://github.com/gee-community/geemap/blob/master/examples/notebooks/11_export_image.ipynb
geemap.ee_export_image_collection(modis, out_dir = out_dir_m)

Total number of images: 35

Exporting 1/35: /home/osboxes/Documents/MSc_Thesis/data/modis/cloudmasked/2014_01_14.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/modis/cloudmasked/2014_01_14.tif


Exporting 2/35: /home/osboxes/Documents/MSc_Thesis/data/modis/cloudmasked/2014_01_15.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/modis/cloudmasked/2014_01_15.tif


Exporting 3/35: /home/osboxes/Documents/MSc_Thesis/data/modis/cloudmasked/2014_01_16.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/modis/cloudmasked/2014_01_16.tif


Exporting 4/35: /home/osboxes/Documents/MSc_Thesis/data/modis/cloudmasked/2014_01_17.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_Thesis/data/modis/cloudmasked/2014_01_17.tif


Exporting 5/35: /home/osboxes/Documents/MSc_Thesis/data/modis/cloudmasked/2014_01_18.tif
Generating 